In [3]:
# Check
import sys, subprocess, pkg_resources, platform, datetime
print("Python version:", platform.python_version())

Python version: 3.11.13


In [4]:
!pip install -q duckdb pandas pyarrow

In [5]:
# Import libraries and quick sanity check
import duckdb, pandas as pd
duckdb.sql("SELECT 'DuckDB is ready' AS status").show()

┌─────────────────┐
│     status      │
│     varchar     │
├─────────────────┤
│ DuckDB is ready │
└─────────────────┘



In [6]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Verify project exists within the folder
!ls /content/drive/MyDrive/restaurant-ai-sql-project

data_raw  outputs


In [8]:
import pathlib

# Path to the CSVs placed in Drive
DATA_PATH = pathlib.Path('/content/drive/MyDrive/restaurant-ai-sql-project/data_raw')

# Read CSVs with pandas
menu   = pd.read_csv(DATA_PATH / 'menu_items.csv')
orders = pd.read_csv(DATA_PATH / 'order_details.csv')

# Register with DuckDB so we can use plain SQL
con = duckdb.connect()
con.register('menu',   menu)
con.register('orders', orders)

In [9]:
con.sql("SELECT * FROM menu   LIMIT 3").show()
con.sql("SELECT * FROM orders LIMIT 3").show()

┌──────────────┬──────────────┬──────────┬────────┐
│ menu_item_id │  item_name   │ category │ price  │
│    int64     │   varchar    │ varchar  │ double │
├──────────────┼──────────────┼──────────┼────────┤
│          101 │ Hamburger    │ American │  12.95 │
│          102 │ Cheeseburger │ American │  13.95 │
│          103 │ Hot Dog      │ American │    9.0 │
└──────────────┴──────────────┴──────────┴────────┘

┌──────────────────┬──────────┬────────────┬─────────────┬─────────┐
│ order_details_id │ order_id │ order_date │ order_time  │ item_id │
│      int64       │  int64   │  varchar   │   varchar   │ double  │
├──────────────────┼──────────┼────────────┼─────────────┼─────────┤
│                1 │        1 │ 1/1/23     │ 11:38:36 AM │   109.0 │
│                2 │        2 │ 1/1/23     │ 11:57:40 AM │   108.0 │
│                3 │        2 │ 1/1/23     │ 11:57:40 AM │   124.0 │
└──────────────────┴──────────┴────────────┴─────────────┴─────────┘



In [10]:
# Row count (union -- keeps output tidy)
con.sql("""
    SELECT 'menu'   AS table_name, COUNT(*) AS rows FROM menu
    UNION ALL
    SELECT 'orders',               COUNT(*)          FROM orders
""").show()

┌────────────┬───────┐
│ table_name │ rows  │
│  varchar   │ int64 │
├────────────┼───────┤
│ menu       │    32 │
│ orders     │ 12234 │
└────────────┴───────┘



In [11]:
# Date range check on the orders table
con.sql("""
    SELECT
        MIN(order_date) AS first_day,
        MAX(order_date) AS last_day
    FROM orders
""").show()

┌───────────┬──────────┐
│ first_day │ last_day │
│  varchar  │ varchar  │
├───────────┼──────────┤
│ 1/1/23    │ 3/9/23   │
└───────────┴──────────┘



In [12]:
# Null checks (price & item references)
con.sql("SELECT COUNT(*) AS missing_prices FROM menu WHERE price IS NULL").show()
con.sql("SELECT COUNT(*) AS orphan_items  FROM orders WHERE item_id IS NULL").show()

┌────────────────┐
│ missing_prices │
│     int64      │
├────────────────┤
│              0 │
└────────────────┘

┌──────────────┐
│ orphan_items │
│    int64     │
├──────────────┤
│          137 │
└──────────────┘



In [13]:
# Preview first 3 rows
con.sql("SELECT * FROM menu   LIMIT 3").show()
con.sql("SELECT * FROM orders LIMIT 3").show()

┌──────────────┬──────────────┬──────────┬────────┐
│ menu_item_id │  item_name   │ category │ price  │
│    int64     │   varchar    │ varchar  │ double │
├──────────────┼──────────────┼──────────┼────────┤
│          101 │ Hamburger    │ American │  12.95 │
│          102 │ Cheeseburger │ American │  13.95 │
│          103 │ Hot Dog      │ American │    9.0 │
└──────────────┴──────────────┴──────────┴────────┘

┌──────────────────┬──────────┬────────────┬─────────────┬─────────┐
│ order_details_id │ order_id │ order_date │ order_time  │ item_id │
│      int64       │  int64   │  varchar   │   varchar   │ double  │
├──────────────────┼──────────┼────────────┼─────────────┼─────────┤
│                1 │        1 │ 1/1/23     │ 11:38:36 AM │   109.0 │
│                2 │        2 │ 1/1/23     │ 11:57:40 AM │   108.0 │
│                3 │        2 │ 1/1/23     │ 11:57:40 AM │   124.0 │
└──────────────────┴──────────┴────────────┴─────────────┴─────────┘



In [14]:
# Build a CTE that merges 'orders' and 'menu'
order_items_cte = """
WITH order_items AS (
    SELECT
        o.order_id,

        -- Parse 'M/D/YY' or 'MM/DD/YY' safely into a DATE
        TRY_STRPTIME(o.order_date, '%m/%d/%y') AS order_date,

        m.menu_item_id               AS item_id,
        m.item_name,
        m.category,
        CAST(m.price AS DOUBLE)      AS price
    FROM orders AS o
    LEFT JOIN menu  AS m
           ON o.item_id = m.menu_item_id
)
"""
# Quick sanity check
con.sql(order_items_cte + "SELECT order_id, order_date, item_name FROM order_items LIMIT 5;").show()

┌──────────┬─────────────────────┬──────────────────┐
│ order_id │     order_date      │    item_name     │
│  int64   │      timestamp      │     varchar      │
├──────────┼─────────────────────┼──────────────────┤
│        1 │ 2023-01-01 00:00:00 │ Korean Beef Bowl │
│        2 │ 2023-01-01 00:00:00 │ Tofu Pad Thai    │
│        2 │ 2023-01-01 00:00:00 │ Spaghetti        │
│        2 │ 2023-01-01 00:00:00 │ Chicken Burrito  │
│        2 │ 2023-01-01 00:00:00 │ Mushroom Ravioli │
└──────────┴─────────────────────┴──────────────────┘



In [15]:
# Top-10 best-selling items with revenue + rank
leaderboard_sql = order_items_cte + """
SELECT
    item_name,
    COUNT(*)                     AS num_sold,
    ROUND(SUM(price), 2)         AS total_revenue,
    RANK() OVER (ORDER BY COUNT(*) DESC) AS sales_rank
FROM order_items
GROUP BY item_name
ORDER BY sales_rank
LIMIT 10;
"""
con.sql(leaderboard_sql).show()

┌───────────────────────┬──────────┬───────────────┬────────────┐
│       item_name       │ num_sold │ total_revenue │ sales_rank │
│        varchar        │  int64   │    double     │   int64    │
├───────────────────────┼──────────┼───────────────┼────────────┤
│ Hamburger             │      622 │        8054.9 │          1 │
│ Edamame               │      620 │        3100.0 │          2 │
│ Korean Beef Bowl      │      588 │       10554.6 │          3 │
│ Cheeseburger          │      583 │       8132.85 │          4 │
│ French Fries          │      571 │        3997.0 │          5 │
│ Tofu Pad Thai         │      562 │        8149.0 │          6 │
│ Steak Torta           │      489 │       6821.55 │          7 │
│ Spaghetti & Meatballs │      470 │        8436.5 │          8 │
│ Mac & Cheese          │      463 │        3241.0 │          9 │
│ Chips & Salsa         │      461 │        3227.0 │         10 │
├───────────────────────┴──────────┴───────────────┴────────────┤
│ 10 rows 

In [16]:
# Revenue vs. estimated margin (assume 40 % cost of goods)
margin_sql = order_items_cte + """
SELECT
    category,
    ROUND(SUM(price),        2) AS revenue,
    ROUND(SUM(price)*0.40,  2) AS est_food_cost,   -- 40 % COGS
    ROUND(SUM(price)*0.60,  2) AS est_margin       -- 60 % margin
FROM order_items
GROUP BY category
ORDER BY est_margin DESC;
"""
con.sql(margin_sql).show()

┌──────────┬──────────┬───────────────┬────────────┐
│ category │ revenue  │ est_food_cost │ est_margin │
│ varchar  │  double  │    double     │   double   │
├──────────┼──────────┼───────────────┼────────────┤
│ Italian  │  49462.7 │      19785.08 │   29677.62 │
│ Asian    │ 46720.65 │      18688.26 │   28032.39 │
│ Mexican  │  34796.8 │      13918.72 │   20878.08 │
│ American │ 28237.75 │       11295.1 │   16942.65 │
│ NULL     │     NULL │          NULL │       NULL │
└──────────┴──────────┴───────────────┴────────────┘



In [17]:
OUTPUT_PATH = pathlib.Path('/content/drive/MyDrive/restaurant-ai-sql-project/outputs')
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

# Full joined transaction table 
order_items_df = con.sql(order_items_cte + "SELECT * FROM order_items").df()
order_items_df.to_csv(OUTPUT_PATH / "order_items.csv", index=False)

# Top-10 leaderboard nleaderboard_df = con.sql(leaderboard_sql).df()
leaderboard_df.to_csv(OUTPUT_PATH / "top10_leaderboard.csv", index=False)

print("CSVs saved to:", OUTPUT_PATH)
print("Files:", list(OUTPUT_PATH.iterdir()))

CSVs saved to: /content/drive/MyDrive/restaurant-ai-sql-project/outputs
Files: [PosixPath('/content/drive/MyDrive/restaurant-ai-sql-project/outputs/leaderboard.csv'), PosixPath('/content/drive/MyDrive/restaurant-ai-sql-project/outputs/top10_leaderboard.csv'), PosixPath('/content/drive/MyDrive/restaurant-ai-sql-project/outputs/order_items.csv'), PosixPath('/content/drive/MyDrive/restaurant-ai-sql-project/outputs/Restaurant Insights.png')]


In [28]:
!pip install -q scikit-learn joblib

In [29]:
# SQL features + label (high-spend = total_spend > $100)
order_features_sql = order_items_cte + """
SELECT
    order_id,

    COUNT(*)                  AS item_count,
    SUM(price)                AS total_spend,

    -- one-hot style counts for cuisine mix
    SUM(CASE WHEN category = 'Italian'  THEN 1 ELSE 0 END) AS italian_cnt,
    SUM(CASE WHEN category = 'American' THEN 1 ELSE 0 END) AS american_cnt,
    SUM(CASE WHEN category = 'Asian'    THEN 1 ELSE 0 END) AS asian_cnt,

    CASE WHEN SUM(price) > 100 THEN 1 ELSE 0 END AS high_spend_flag
FROM order_items
GROUP BY order_id
"""
feats_df = con.sql(order_features_sql).df()
feats_df.head()

,order_id,item_count,total_spend,italian_cnt,american_cnt,asian_cnt,high_spend_flag
0,2,5,64.45,2.0,1.0,1.0,0
1,5,1,12.95,0.0,0.0,0.0,0
2,9,9,132.25,4.0,0.0,2.0,1
3,10,2,22.50,1.0,1.0,0.0,0
4,11,4,45.85,0.0,3.0,1.0,0


In [38]:
# Train/test split
from sklearn.model_selection import train_test_split

X = feats_df.drop(columns=['order_id', 'high_spend_flag', 'total_spend'])
y = feats_df['high_spend_flag']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

In [39]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# Pipeline: 1) replace NaNs with 0  2) fit logistic model
clf = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('logreg',  LogisticRegression(max_iter=1000))
])

clf.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer(fill_value=0, strategy='constant')),
                ('logreg', LogisticRegression(max_iter=1000))])

In [44]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = clf.predict(X_test)

print("Accuracy: {:.1f}%".format(accuracy_score(y_test, y_pred)*100))
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification report:\n", classification_report(y_test, y_pred, digits=2))

Accuracy: 99.8%

Confusion matrix:
 [[1052    1]
 [   1   20]]

Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1053
           1       0.95      0.95      0.95        21

    accuracy                           1.00      1074
   macro avg       0.98      0.98      0.98      1074
weighted avg       1.00      1.00      1.00      1074



In [45]:
# Save model and metrics
import joblib, json, datetime, pathlib

OUTPUT_PATH = pathlib.Path('/content/drive/MyDrive/restaurant-ai-sql-project/outputs')
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

# 1) model
joblib.dump(clf, OUTPUT_PATH / 'high_spend_clf.joblib')

# 2) simple metrics json
metrics = {
    "accuracy": accuracy_score(y_test, y_pred),
    "timestamp": datetime.datetime.utcnow().isoformat()
}
with open(OUTPUT_PATH / 'model_metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("Model + metrics saved to", OUTPUT_PATH)

Model + metrics saved to /content/drive/MyDrive/restaurant-ai-sql-project/outputs
